In [ ]:
%load_ext autoreload
%autoreload 2
import os
os.chdir("/scratch/ewalt/pdm/rs-uncertainty")
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from src.metrics import StratifiedRCU
from src.viz import *
from pathlib import Path
import rasterio
import fiona
from datetime import datetime
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import trange
import random
import yaml
import fiona
import rasterio.warp
import rasterio.features
sns.set()
sns.set_style("whitegrid")
random.seed(123)

RESDIR = "results/cloud_exp/2023-06-20_16-14-11" #"results/cloud_exp/2023-05-26_15-55-46"
S2DIR = "gee_data/reprojected/"
S2REPRDIR = "gee_data/reprojected_dirs"
GTDIR = "assets/data/preprocessed"
SANITYRESDIR = "results/cloud_exp/2023-05-31_11-23-56_sanity_check" # results
SANITYS2DIR = "assets/data/sentinel_data/s2_reprojected" # s2 reprojected
SANITYS2REPRDIR = "gee_data/sanity_check/" # restructured s2 reprojected
SPLITMASKDIR = "assets/data/split_masks/" # split masks
SHAPEFILES = ['assets/data/NHM_projectDekning_AOI_edit2015_V2.shp', 'assets/data/ALS_projects_Dz_all_norway.shp']
STATSFILE = "data/2023-04-05_18-58-33_baseline/stats.yaml"
EMPIRICAL_CP_THRESHOLD = 7.9

with open(STATSFILE, "r") as f:
    stats = yaml.safe_load(f)
TRAINMEANS = stats["labels_stats"]["mean"]
TRAINSTDS = stats["labels_stats"]["std"]
for i in [2,4]:
    TRAINMEANS[i] /= 100
    TRAINSTDS[i] /= 100
    
VARIABLES = ['P95', 'MeanH', 'Dens', 'Gini', 'Cover']

# Experiment result directories
result_dirs = [p.path for p in os.scandir(RESDIR) if os.path.exists(os.path.join(p.path,"rcu.json"))]
outliers = [os.path.join(RESDIR, f"1023_{d}") for d in [
    "20180503T104019", # index: 3, avgcp: 42, all white
    "20180620T105031", # index: 6, avgcp: 0., all white
]]
result_dirs = [r for r in result_dirs if not any(r.__contains__(o) for o in outliers)]
len(result_dirs)

In [ ]:
visualizer, nnz_result_dirs, exp_vars = get_nonzero_avg_cp_visualizer(
    result_dirs, S2REPRDIR, VARIABLES
)

In [ ]:
bins=np.array([0, EMPIRICAL_CP_THRESHOLD, 25, 60, 100])
cloudy_visualizer, nnz_result_dirs, cloudy_exp_vars = get_nonzero_avg_cp_visualizer(
    result_dirs, S2REPRDIR, VARIABLES, max_n=None, bins=bins
)

In [ ]:
cloudless_visualizer = getCloudlessVisualizer(
    nnz_result_dirs,
    GTDIR, 
    S2REPRDIR,
    cloudy_pixel_threshold=EMPIRICAL_CP_THRESHOLD,
    cloudy_exp_vars=cloudy_visualizer.exp_vars,
    variable_names=VARIABLES,
    exp_var_name=cloudy_visualizer.exp_var_name
)

In [ ]:
def showCCMetrics(
    cloudy_visualizer, 
    cloudless_visualizer, 
    bins, metrics, 
    kind="agg", 
    plot_kind="lineplot"
):
    prev_vg = cloudy_visualizer.variable_names
    variable_groups = [["Dens", "Gini"], ["P95", "MeanH", "Cover"]]
    for vg in variable_groups:
        cloudy_visualizer.variable_names = vg
        cloudless_visualizer.variable_names = vg
        showCloudyCloudlessMetrics(
            cloudy_visualizer, 
            cloudless_visualizer,
            bins=bins, 
            metrics=metrics, 
            kind=kind,
            ncols=len(vg),
            figsize=(12,4),
            # save_name=f"images/cloud_experiment/cloudy_cloudless_{plot_kind}_{kind}_{'-'.join(vg)}",
            plot_kind=plot_kind
        )
    cloudy_visualizer.variable_names = prev_vg
    cloudless_visualizer.variable_names = prev_vg

In [ ]:
def showDefenseCCMetrics(
    cloudy_visualizer, 
    cloudless_visualizer, 
    bins, metrics, 
    kind="agg", 
    plot_kind="lineplot"
):
    prev_vg = cloudy_visualizer.variable_names
    variable_groups = [[v] for v in VARIABLES]
    for vg in variable_groups:
        cloudy_visualizer.variable_names = vg
        cloudless_visualizer.variable_names = vg
        showCloudyCloudlessMetrics(
            cloudy_visualizer, 
            cloudless_visualizer,
            bins=bins, 
            metrics=metrics, 
            kind=kind,
            ncols=1,
            figsize=(6.7,5.3),
            save_name=f"images/defense/cloudy_cloudless_{plot_kind}_{kind}_{vg[0]}",
            plot_kind=plot_kind,
            add_title=False
        )
    cloudy_visualizer.variable_names = prev_vg
    cloudless_visualizer.variable_names = prev_vg

In [ ]:
# report plots: boxplot with 4 bins
showCCMetrics(
    cloudy_visualizer, 
    cloudless_visualizer, 
    bins, metrics=["rmse", "srp", "uce", "ence"], 
    kind="agg", 
    plot_kind="boxplot"
)

In [ ]:
showDefenseCCMetrics(
    cloudy_visualizer, 
    cloudless_visualizer, 
    bins, metrics=["rmse", "srp", "uce", "ence"], 
    kind="agg", 
    plot_kind="boxplot",
)

## Confounding variables

In [ ]:
confounding_dirs = [
    os.path.join(RESDIR, "1023_20180526T105029"), # ideal
    os.path.join(RESDIR, "1023_20180418T104021"), #snow
    os.path.join(RESDIR, "1023_20180920T104019"), #shadow
    os.path.join(RESDIR, "1023_20180426T105029"), #snow, shadow and clouds
]
confounding_titles = [
    "ideal",
    "snow",
    "shadow",
    "snow, shadow and clouds"
]
confounder_islice = slice(1120,1250)
confounder_jslice = slice(380,530)

In [ ]:
showRGB(confounding_dirs, S2REPRDIR, 
        confounding_titles, islice=confounder_islice, jslice=confounder_jslice, draw_bbox=True,
        color="r",
        save_name="images/cloud_experiment/counfounders_rgb_complete",
        show_avg_cp=True)
showRGB(confounding_dirs, S2REPRDIR, 
        confounding_titles, islice=confounder_islice, jslice=confounder_jslice, draw_bbox=False,
        color="r",
        save_name="images/cloud_experiment/counfounders_rgb_cropped",
        show_avg_cp=True)

In [ ]:
# Quasi Calibration Maps
rerror_bounds = [15,11,0.5,0.18,0.55] # regression residuals
cerror_pu_bounds = [9, 6.5, 0.35, 0.12, 0.42] # calibration residuals, pu, absolute regression residuals
kwargs = dict(
    predictive_uncertainty_bounds=cerror_pu_bounds,
    rerror_bounds=rerror_bounds,
    cerror_bounds=cerror_pu_bounds
)
# MH, Gini => R, U, C rasters for readability
for varname in VARIABLES:
    varindex = VARIABLES.index(varname)+1
    showQuasiCalibrationMaps(
        confounding_dirs, 
        confounding_titles,
        varindex, 
        varname,
        S2REPRDIR,
        GTDIR,
        SHAPEFILES,
        islice=confounder_islice,
        jslice=confounder_jslice,
        normalize=False,
        figsize=(15,13),
        save_name=f"images/cloud_experiment/quasi_calibration_maps_{varname}",
        **kwargs
    )